In [1]:
require(tidyverse)
require(data.table)
require(forestplot)
require(meta)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: forestplot
Loading required package: grid
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: checkmate
Loading required package: meta
Loadi

In [2]:
# madata <- data.frame(
#     Variant = c('Gln175His', 'Arg220Cys'),
#     TE      = c(log(0.661179), -0.3335), 
#     seTE    = c(0.148816,       0.0546),
#     stringsAsFactors=F
# )


In [3]:
# m<-metagen(
#     TE,
#     seTE,
#     data=madata,
#     studlab=paste(Variant),
#     comb.fixed = TRUE,
#     comb.random = FALSE,
#     prediction=TRUE,
#     sm="SMD"
# )

In [4]:
# m

In [5]:
# m$pval.fixed

In [6]:
# pooled_TE   = m$TE.fixed
# pooled_seTE = m$seTE.fixed

In [7]:
# tabletext<-cbind(
#     c('Variant',    madata %>% pull(Variant), 'Summary'),
#     c('BETA', madata %>% pull(TE) %>% lapply(round, digits =2) %>% lapply(as.character) %>% 
#       simplify(), as.character(round(pooled_TE, digits=2)))
# ) 


In [8]:
# plot_data <- bind_rows(
#     data.frame(TE=NA, seTE=NA), 
#     madata, 
#     data.frame(TE=pooled_TE, seTE=pooled_seTE)
# ) %>%
# rename(mean = TE) %>%
# mutate(
#     lower = mean - 1.96 * seTE,
#     upper = mean + 1.96 * seTE
# ) %>%
# select(mean, lower, upper)

In [9]:
# png(file='ANGPTL7_Glaucoma_BETA_forest.png', width=1600, height=1200, res=300) 
# forestplot(
#     tabletext, 
#     plot_data,
#     new_page = TRUE,
#     is.summary=c(TRUE,rep(FALSE,madata %>% dim() %>% first()),TRUE),
#     xlog=FALSE, 
#     col=fpColors(box="royalblue",line="darkblue", summary="royalblue")
# )
# dev.off()

In [10]:
madata_full <- data.frame(
    Variant = c('Gln175His', 'Arg177Ter', 'Arg140His', 'Gln136Arg', 'Arg220Cys'),
    TE      = c(log(0.661179), log(0.582775), log(0.917626), log(0.210271), -0.3335), 
    seTE    = c(     0.148816,      0.711997,      0.226641,       1.00283,  0.0546),
    stringsAsFactors=F
)


In [11]:
m_full<-metagen(
    TE,
    seTE,
    data=madata_full,
    studlab=paste(Variant),
    comb.fixed = TRUE,
    comb.random = FALSE,
    prediction=TRUE,
    sm="SMD"
)

In [12]:
m_full

              SMD             95%-CI %W(fixed)
Gln175His -0.4137 [-0.7054; -0.1221]      11.2
Arg177Ter -0.5400 [-1.9354;  0.8555]       0.5
Arg140His -0.0860 [-0.5302;  0.3582]       4.8
Gln136Arg -1.5594 [-3.5249;  0.4062]       0.2
Arg220Cys -0.3335 [-0.4405; -0.2265]      83.2

Number of studies combined: k = 5

                        SMD             95%-CI     z  p-value
Fixed effect model  -0.3346 [-0.4322; -0.2369] -6.72 < 0.0001
Prediction interval         [-0.4931; -0.1760]               

Quantifying heterogeneity:
tau^2 = 0; H = 1.00 [1.00; 1.92]; I^2 = 0.0% [0.0%; 72.8%]

Test of heterogeneity:
    Q d.f. p-value
 3.06    4  0.5476

Details on meta-analytical method:
- Inverse variance method

In [14]:
m_full$pval.fixed %>% print()
pooled_TE_full   = m_full$TE.fixed
pooled_seTE_full = m_full$seTE.fixed

[1] 1.859792e-11


In [29]:
summary_TE_str <- paste0(
    as.character(round(exp(pooled_TE_full), digits=2)),
    ' [', 
    as.character(round(exp(pooled_TE_full - 1.96 * pooled_seTE_full), digits=2)),
    ', ',
    as.character(round(exp(pooled_TE_full + 1.96 * pooled_seTE_full), digits=2)),
    ']'
)
                 

In [68]:
tabletext_full<-cbind(
    c('Variant',    madata_full %>% pull(Variant), 'Summary'),
    c('OR [95% CI]', madata_full %>% pull(TE) %>% lapply(exp) %>%
      lapply(round, digits =2) %>% lapply(as.character) %>% simplify(), 
      summary_TE_str)
) 
plot_data_full <- bind_rows(
    data.frame(TE=NA, seTE=NA), 
    madata_full, 
    data.frame(TE=pooled_TE_full, seTE=pooled_seTE_full)
) %>%
rename(mean = TE) %>%
mutate(
    lower = mean - 1.96 * seTE,
    upper = mean + 1.96 * seTE
) %>%
select(mean, lower, upper)

In [73]:
png(file='ANGPTL7_Glaucoma_BETA_forest_full.png', width=1600, height=1200, res=300) 
forestplot(
    tabletext_full, 
    plot_data_full,
    new_page = TRUE,
    is.summary=c(TRUE,rep(FALSE,madata_full %>% dim() %>% first()),TRUE),
    xlog=FALSE, 
    col=fpColors(box="royalblue",line="darkblue", summary="royalblue"),
    xticks = c(-log(32), -log(16), -log(8), -log(4), -log(2), -log(1), log(2))
)
dev.off()

png 
  2

In [42]:
# tabletext_full_OR<-cbind(
#     c('Variant',    madata_full %>% pull(Variant), 'Summary'),
#     c('OR', madata_full %>% pull(TE) %>% lapply(exp) %>% 
#       lapply(round, digits =2) %>% lapply(as.character) %>% simplify(), 
#       as.character(round(exp(pooled_TE), digits=2)))
# ) 
# plot_data_full_OR <- bind_rows(
#     data.frame(TE=NA, seTE=NA), 
#     madata_full, 
#     data.frame(TE=pooled_TE_full, seTE=pooled_seTE_full)
# ) %>%
# rename(mean = TE) %>%
# mutate(
#     lower = exp(mean - 1.96 * seTE),
#     upper = exp(mean + 1.96 * seTE),
#     mean = exp(mean)
# ) %>%
# select(mean, lower, upper)

In [43]:
# # png(file='ANGPTL7_Glaucoma_BETA_forest_full_OR.png', width=1600, height=1200, res=300) 
# forestplot(
#     tabletext_full_OR, 
#     plot_data_full_OR,
#     new_page = TRUE,
#     is.summary=c(TRUE,rep(FALSE,madata_full %>% dim() %>% first()),TRUE),
#     xlog=FALSE, 
#     col=fpColors(box="royalblue",line="darkblue", summary="royalblue")
# )
# # dev.off()

png 
  2

## reference 
- https://bookdown.org/MathiasHarrer/Doing_Meta_Analysis_in_R/fixed.html
- https://cran.r-project.org/web/packages/forestplot/vignettes/forestplot.html

In [78]:
exp(-2.079)

[1] 0.1250552